### 1. Получаем данные из Surechembl db

In [ ]:
import duckdb, pyarrow.parquet as pq

OUT_PARQUET = "./output/pharma_patents_2020.parquet"
DUCKDB_FILE = "/home/vshepard/hackaton_life/surecheml_db.duckdb"

q = f"""
COPY (
    WITH pharma_cpc AS (
        SELECT DISTINCT id
        FROM patents, UNNEST(cpc) t(code)
        WHERE trim(code) LIKE 'A61K%' OR trim(code) LIKE 'A61P%'
    ), pharma_ipc AS (
        SELECT DISTINCT id
        FROM patents, UNNEST(ipc) t(code)
        WHERE trim(code) LIKE 'A61K%' OR trim(code) LIKE 'A61P%'
    ), pharma_ids AS (
        SELECT id FROM pharma_cpc
        UNION
        SELECT id FROM pharma_ipc
    )
    SELECT DISTINCT
           p.patent_number,
           c.id          AS compound_id,
           c.smiles,
           c.inchi_key
    FROM   pharma_ids  pid
    JOIN   patents     p   ON p.id = pid.id and p.publication_date >= '2020-01-01'
    JOIN   patent_compound_map pcm ON pcm.patent_id = p.id
    JOIN   compounds   c   ON c.id = pcm.compound_id AND c.smiles <> 'N' and c.smiles IS NOT NULL
    JOIN   fields      f   ON f.id = pcm.field_id and f.fieldname = 'desc'
)
TO '{OUT_PARQUET}'
WITH (FORMAT PARQUET, COMPRESSION 'ZSTD');
"""

duckdb.connect(DUCKDB_FILE).execute(q)
print("Parquet written →", OUT_PARQUET, "| rows =", pq.ParquetFile(OUT_PARQUET).metadata.num_rows)

In [1]:
# check step 1
import duckdb

file = "./output/pharma_patents_2020.parquet"

query = f"""
    SELECT count(1)
    FROM '{file}';
    """
res = duckdb.query(query).fetchall()
print(res)

[(131538147,)]


### Отфильтровать статьи с данными измерений IC50|Ki|EC50|Kd

In [2]:
import duckdb
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import re

file = "./output/pharma_patents_2020.parquet"
output_csv = "./output/patents_with_ic50_ki_ec50.csv"

# 1. Get all unique patent numbers (already deduplicated)
query = f"SELECT patent_number, list(smiles) AS smiles_array FROM '{file}' group by 1 limit 10;"
patent_rows = duckdb.query(query).fetchall()

def process_patent(patent_number):
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)
    url = f"https://www.surechembl.org/patent/{patent_number}"
    found = None
    try:
        driver.get(url)
        time.sleep(6)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        description_section = None
        # Extract description
        for el in soup.find_all(class_=lambda c: c and 'description' in c):
            sec = el.find(class_='section')
            if sec:
                description_section = sec.get_text(separator=" ", strip=True)
                break

        # Check for bioactivity keywords: ('Ki (nM)', 'IC50 (nM)', 'Kd (nM)', 'EC50 (nM)')
        if description_section and re.search(r'\b(IC50|Ki|EC50|Kd|pIC50|pKi|pKd)\b', description_section, re.IGNORECASE):
            found = description_section
    except Exception as e:
        print(f"Error with patent {patent_number}: {e}")
    finally:
        driver.quit()

    return found

In [ ]:
SYSTEM_PROMPT = """You are ChemExtract‑GPT, a meticulous chemoinformatics extractor.
Output ONLY CSV lines and NOTHING else.
Columns (in order):
latent_number,Ligand,SMILES,InChIKey,Protein Name,UniProt,Affinity Type,Affinity (nM),Temp (°C),pH,Assay Notes
Rules:
1. Each line must correspond to one quantitative binding measurement (Ki, IC50, Kd, EC50).
2. Convert any pX values to nanomolar:  IC50(M) = 10^(−pIC50); multiply by 1e9 to get nM.
3. Convert μM, mM, pM → nM.
4. “Protein Name” must be the explicit target name appearing in the text; map to the best UniProt accession if possible (else leave empty).
5. Do **not** emit commentary, explanations, Markdown, or extra whitespace — only the CSV header once and data lines. If there is no required binding measurement (Ki, IC50, Kd, EC50) write nothing.
"""

### Поиск значений измерений

In [4]:
import csv, json, logging, shelve, time, requests
from typing import List, Dict
import pandas as pd
from tqdm import tqdm

# ------------------------------------------------------------------------------
# CONSTANTS
# ------------------------------------------------------------------------------
CACHE_PATH   = "./output/patent_html_cache.db"
OUT_CSV      = "./output/bindingdb_upload.csv"
LLM_MODEL    = "llama-3.3-70b-instruct"
OPENAI_API_KEY  = "dummy-key"
OPENAI_BASE_URL = "http://80.209.242.40:8000/v1"
TEMPERATURE  = 0.25
MAX_TOKENS   = 2048
CSV_HEADER = ["Patent","LigandID","SMILES","InChIKey","Protein Name","UniProt",
              "Affinity Type","Affinity (nM)","Temp (°C)","pH","Assay Notes"]

SYSTEM_PROMPT = """
You are BioFilter-GPT, an expert in patent text mining for biochemical data extraction.

Your job is to **extract only those portions of the text** that are likely to contain information needed to fill the following CSV fields:

Patent, LigandID, SMILES, InChIKey, Protein Name, UniProt, Affinity Type, Affinity (nM), Temp (°C), pH, Assay Notes

**Relevance rules:**
- Keep any sentence, paragraph, or table mentioning binding affinity values (e.g., Ki, IC50, EC50, Kd).
- Keep text that mentions experimental conditions, such as temperature, pH, buffers, solvents, or assay method.
- Keep text describing target proteins or their identifiers (gene name, Uniprot accession, etc.).
- Keep text describing the compound, such as SMILES, InChI, or structure-related data.
- Discard introductions, unrelated background, legal boilerplate, patent claims that do not reference measurements or proteins, etc.

**Output only the minimal necessary text fragments** (preferably in their original order) that could be useful for structured extraction.
Do not summarize, paraphrase, or add any commentary.

If no relevant content is found, return an empty string.

---
"""

USER_TEMPLATE = """
Below is the full text of a chemical patent.
**Extract and return only the sentences, paragraphs, or tables** likely to contain:
- Quantitative binding measurements (Ki, IC50, Kd, EC50)
- Descriptions of the experiment (temperature, pH, buffer, detection method, cell line, etc.)
- Protein targets or gene identifiers
- Compound identifiers (SMILES, InChI, InChIKey)

Do NOT include irrelevant patent sections, background, or legal text.
----- BEGIN PATENT TEXT -----
{FULL_PATENT_TEXT}
----- END PATENT TEXT -----
"""

SYSTEM_PROMPT2 = """
You are ChemExtract-GPT, a meticulous chemoinformatics extractor.

Your task is to extract structured biochemical binding data from chemical patent text and output it as CSV lines with these columns, in order:

Patent,LigandID,SMILES,InChIKey,Protein Name,UniProt,Affinity Type,Affinity (nM),Temp (°C),pH,Assay Notes

**Instructions:**
- Each line must correspond to one quantitative binding measurement (Ki, IC50, Kd, EC50, pKi, pIC50, pKd, pEC50).
- For pIC50, pKi, or pKd, convert to nM using: IC50(M) = 10^(−pIC50); multiply by 1e9 to get nM.
- Convert all values to nM (μM × 1e3, mM × 1e6, pM × 1e-3).
- “Protein Name” must be the explicit target name.
- Fill "Assay Notes" with up to 255 characters describing buffer, detection method, or cell line if available.
- Omit duplicate (Patent, LigandID, Protein, Affinity Type, Affinity) pairs.
- Output **ONLY** the CSV header (once) and data lines—no extra commentary, Markdown, or whitespace.

---
"""

USER_TEMPLATE2 = """
Below is a cleaned excerpt from a chemical patent, plus relevant identifiers.

Patent number: {PATENT_NUMBER}
SMILES: {SMILES}

----- BEGIN RELEVANT PATENT TEXT -----
{FULL_PATENT_TEXT}
----- END RELEVANT PATENT TEXT -----

**Extract and output all relevant binding measurements and assay details in the requested CSV format.**
"""
# ------------------------------------------------------------------------------
# SESSION (all headers in one place)
# ------------------------------------------------------------------------------
session = requests.Session()
session.headers.update({
    "User-Agent":  "ChemExtract/0.1 (+https://github.com/you)",
    "Content-Type": "application/json",
    "Authorization": "Bearer {}".format(OPENAI_API_KEY)
})

# ------------------------------------------------------------------------------
# HELPER FUNCTIONS
# ------------------------------------------------------------------------------
def build_messages(row: pd.Series, text: str, user_temp, system_prompt) -> List[Dict[str, str]]:
    user_msg = user_temp.format(
        PATENT_NUMBER=row[0],
        SMILES=row[1],
        FULL_PATENT_TEXT=text
    )
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": user_msg}
    ]


def call_llama(messages: List[Dict[str, str]]) -> str:
    payload = {
        "model": LLM_MODEL,
        "messages": messages,
        "max_tokens": MAX_TOKENS,
        "temperature": TEMPERATURE
    }
    resp = session.post("{}/chat/completions".format(OPENAI_BASE_URL),
                        json=payload, timeout=90)
    resp.raise_for_status()
    return resp.json()["choices"][0]["message"]["content"]


def parse_csv_response(csv_text: str) -> List[List[str]]:
    lines = [l.strip() for l in csv_text.strip().splitlines() if l.strip()]
    if lines and lines[0].lower().startswith("patent,"):
        lines = lines[1:]
    return [next(csv.reader([line])) for line in lines]


def split_text_for_llm(text: str, max_length: int = 4000) -> List[str]:
    """Greedy paragraph splitter that respects max_length."""
    if len(text) <= max_length:
        return [text]

    paras, chunks, current = text.split("\n\n"), [], ""
    for p in paras:
        if len(current) + len(p) + 2 <= max_length:
            current += p + "\n\n"
        else:
            if current:
                chunks.append(current.rstrip())
            current = p + "\n\n"
    if current:
        chunks.append(current.rstrip())
    return chunks


# ------------------------------------------------------------------------------
# MAIN DRIVER
# ------------------------------------------------------------------------------
def process_llm():
    logging.info("Processing {} patents".format(len(patent_rows)))

    with open(OUT_CSV, "w", newline="", encoding="utf-8") as out_fh:
        writer = csv.writer(out_fh)
        writer.writerow(CSV_HEADER)

        for row in patent_rows:
            # 1) get full text (cached)
            full_text = process_patent(row[0])
            if not full_text:
                continue
            # 2) chunk if necessary
            text = ""
            for chunk in split_text_for_llm(full_text, max_length=4000):
                messages1 = build_messages(row, chunk, USER_TEMPLATE, SYSTEM_PROMPT)
                text += call_llama(messages1)

            messages2 = build_messages(row, text, USER_TEMPLATE2, SYSTEM_PROMPT2)
            llm_raw = call_llama(messages2)

            # 3) parse and write
            for csv_row in parse_csv_response(llm_raw):
                writer.writerow(csv_row)


process_llm()


Error with patent US-10954289-B1: HTTPConnectionPool(host='localhost', port=40565): Read timed out. (read timeout=120)
